In [20]:
import statsmodels.api as sm
import statsmodels.stats.sandwich_covariance as sw
import numpy as np
import pandas as pd

## Import data

In [ ]:
OK = pd.read_excel(
    "data/OK.xlsx",
    sheet_name=0,
    engine="openpyxl"
)

OK["Date"] = pd.to_datetime(OK["Date"])
OK = OK.set_index("Date").sort_index()

OK.head()

,Close
Date,
2020-01-02,54.99
2020-01-03,54.69
2020-01-06,54.67
2020-01-07,54.25
2020-01-08,54.35


In [43]:
SP500 = pd.read_excel(
    "data/SP500.xlsx",
    sheet_name=0,
    engine="openpyxl"
)

SP500["Date"] = pd.to_datetime(SP500["Date"])
SP500 = SP500.set_index("Date").sort_index()

SP500.head()

,Close
Date,
2020-01-02,3257.85
2020-01-03,3234.85
2020-01-06,3246.28
2020-01-07,3237.18
2020-01-08,3253.05


## Calculate daily rate of return

In [44]:
# r(t+1) = ln[P(t+1) / P(t)]
OK["log_return"] = np.log(OK["Close"] / OK["Close"].shift(1))

# r(t+1) = [P(t+1) - P(t)] / P(t)
OK["arith_return"] = OK["Close"].pct_change()

OK.head()

,Close,log_return,arith_return
Date,,,
2020-01-02,54.99,NaN,NaN
2020-01-03,54.69,-0.005470,-0.005456
2020-01-06,54.67,-0.000366,-0.000366
2020-01-07,54.25,-0.007712,-0.007682
2020-01-08,54.35,0.001842,0.001843


In [45]:
# r(t+1) = ln[P(t+1) / P(t)]
SP500["log_return"] = np.log(SP500["Close"] / SP500["Close"].shift(1))

# r(t+1) = [P(t+1) - P(t)] / P(t)
SP500["arith_return"] = SP500["Close"].pct_change()

SP500.head()

,Close,log_return,arith_return
Date,,,
2020-01-02,3257.85,NaN,NaN
2020-01-03,3234.85,-0.007085,-0.007060
2020-01-06,3246.28,0.003527,0.003533
2020-01-07,3237.18,-0.002807,-0.002803
2020-01-08,3253.05,0.004890,0.004902


## Stock data

In [ ]:
# 选用对数收益率列并去掉 NaN
KO_ret = OK["log_return"].dropna()

# 计算统计量
stats = {
    "Name"       : "Coca-Cola",
    "Ticker"     : "KO",
    "Mean"       : KO_ret.mean(),
    "Median"     : KO_ret.median(),
    "Std Dev"    : KO_ret.std(ddof=1),   # 样本标准差
    "Min"        : KO_ret.min(),
    "Max"        : KO_ret.max(),
    "Skewness"   : KO_ret.skew(),
    "Kurtosis"   : KO_ret.kurt(),
    "Start Date" : KO_ret.index.min().date(),
    "End Date"   : KO_ret.index.max().date()
}


summary = pd.DataFrame(stats, index=["Value"]).T
print(summary.to_string())

                 Value
Name         Coca-Cola
Ticker              KO
Mean          0.000193
Median        0.000958
Std Dev       0.015624
Min          -0.101728
Max           0.062783
Skewness     -0.841427
Kurtosis      6.966093
Start Date  2020-01-03
End Date    2022-12-30


In [ ]:
# 选用对数收益率列并去掉 NaN
SP500_ret = SP500["log_return"].dropna()

# 计算统计量
stats = {
    "Name"       : "S&P 500",
    "Ticker"     : "^GSPC",
    "Mean"       : SP500_ret.mean(),
    "Median"     : SP500_ret.median(),
    "Std Dev"    : SP500_ret.std(ddof=1),   # 样本标准差
    "Min"        : SP500_ret.min(),
    "Max"        : SP500_ret.max(),
    "Skewness"   : SP500_ret.skew(),
    "Kurtosis"   : SP500_ret.kurt(),
    "Start Date" : SP500_ret.index.min().date(),
    "End Date"   : SP500_ret.index.max().date()
}

summary = pd.DataFrame(stats, index=["Value"]).T
print(summary.to_string())

                 Value
Name           S&P 500
Ticker           ^GSPC
Mean          0.000218
Median        0.000881
Std Dev       0.016118
Min          -0.127652
Max           0.089683
Skewness     -0.745384
Kurtosis     10.943675
Start Date  2020-01-03
End Date    2022-12-30


## OLS

In [13]:
#--y = np.asarray(OK['Close']).astype(float)
X = pd.to_datetime(OK['Date']).apply(lambda x: x.toordinal()) 
X = np.asarray(sm.add_constant(X))

In [14]:
model = sm.OLS(OK['Close'], X).fit()

In [15]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     1217.
Date:                Mon, 16 Jun 2025   Prob (F-statistic):          1.83e-159
Time:                        22:13:31   Log-Likelihood:                -2053.0
No. Observations:                 756   AIC:                             4110.
Df Residuals:                     754   BIC:                             4119.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.079e+04    311.054    -34.704      0.0

In [16]:
hac_cov = sw.cov_hac(model, 1)

In [17]:
s = model.get_robustcov_results(cov_type='HAC', maxlags=1)
print(s.summary())

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     333.7
Date:                Mon, 16 Jun 2025   Prob (F-statistic):           5.26e-62
Time:                        22:13:42   Log-Likelihood:                -2053.0
No. Observations:                 756   AIC:                             4110.
Df Residuals:                     754   BIC:                             4119.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.079e+04    594.031    -18.172      0.0